In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')
import seaborn as sns

In [2]:
# !pip install yellowbrick

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot

In [4]:
history_data = pd.read_csv(os.path.join('Stage2_NCAA_2013T02018.csv'))
history_data.head()

,Season,DayNum,outcome,TeamID,Score,NumOT,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2013,4,1,1104,70,0,25,52,6,21,14,14,8,19,14,12,8,4,16
1,2013,4,1,1107,69,0,28,61,7,19,6,7,14,21,13,12,6,2,22
2,2013,4,1,1114,84,0,28,61,6,17,22,30,11,28,16,14,9,4,19
3,2013,4,1,1116,73,0,22,51,3,13,26,32,7,16,16,9,9,8,14
4,2013,4,1,1120,61,0,22,54,3,11,14,21,12,30,10,16,10,3,17


In [5]:
#To check if the dataset is missing values
# sns.heatmap(history_data.isnull(),yticklabels=False,cbar=False,cmap='viridis')
#no nulls found here in this dataset

In [6]:
# sns.set_style('whitegrid')
# sns.countplot(x='outcome', hue='Season', data=history_data, palette='RdBu_r')

In [7]:
history_data.isnull().sum()

Season     0
DayNum     0
outcome    0
TeamID     0
Score      0
NumOT      0
FGM        0
FGA        0
FGM3       0
FGA3       0
FTM        0
FTA        0
OR         0
DR         0
Ast        0
TO         0
Stl        0
Blk        0
PF         0
dtype: int64

In [8]:
history_data.columns

Index(['Season', 'DayNum', 'outcome', 'TeamID', 'Score', 'NumOT', 'FGM', 'FGA',
       'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk',
       'PF'],
      dtype='object')

In [9]:
# Assign X (data) and y (target)
X = history_data.drop("outcome", axis=1)
y = history_data["outcome"]
print(X.shape, y.shape)

(64410, 18) (64410,)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [11]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

/Users/selviramalingam/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [12]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/selviramalingam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
/Users/selviramalingam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


In [13]:
# Create a support vector machine linear classifer and fit it to the training data
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [14]:
# Print the model score using the test data
print("test score: %.3f" % model.score(X_test_scaled,y_test))
predictions = model.predict(X_test_scaled)

test score: 0.886


In [15]:
# Calculate the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=['Loss', 'Won']))

              precision    recall  f1-score   support

        Loss       0.88      0.89      0.89      8052
         Won       0.89      0.88      0.89      8051

   micro avg       0.89      0.89      0.89     16103
   macro avg       0.89      0.89      0.89     16103
weighted avg       0.89      0.89      0.89     16103



In [16]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
#classifier.fit(X_train, y_train)
classifier.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.8834330428302316
Testing Data Score: 0.8856734769918648


In [19]:
predictions = classifier.predict(X_test_scaled)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1 0 1 0 1 1 0 0 1 0]
First 10 Actual labels: [1, 0, 0, 0, 1, 1, 0, 0, 1, 1]


In [20]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1,1
1,0,0
2,1,0
3,0,0
4,1,1
5,1,1
6,0,0
7,0,0
8,1,1
9,0,1


In [21]:
temp = pd.DataFrame()

In [22]:
output_file="prediction2013_2018.csv"

In [23]:
temp['prediction'] = predictions
temp['actual'] = y_test.tolist()
temp["TeamID"]=X_test["TeamID"].tolist()

In [24]:
temp.head()

,prediction,actual,TeamID
0,1,1,1233
1,0,0,1155
2,1,0,1316
3,0,0,1235
4,1,1,1335


In [25]:
temp.to_csv(output_file,index=False)

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8530708563621685

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.8528224554430851

In [28]:
sorted(zip(rf.feature_importances_, X), reverse=True)

[(0.1468906685408217, 'Score'),
 (0.12396424261638007, 'DR'),
 (0.08866446846306072, 'PF'),
 (0.08046439657268348, 'FGA'),
 (0.07203739397650827, 'Ast'),
 (0.054902495532639393, 'FTM'),
 (0.05461749045330172, 'FGM'),
 (0.04645501124720067, 'TO'),
 (0.04457413157891022, 'FTA'),
 (0.04453700029943399, 'Stl'),
 (0.04085920680800168, 'TeamID'),
 (0.03944705363203575, 'FGA3'),
 (0.03809152608410903, 'DayNum'),
 (0.033999185139398834, 'OR'),
 (0.03246963711687371, 'Blk'),
 (0.02745344309662211, 'FGM3'),
 (0.020875958742251437, 'Season'),
 (0.009696690099767266, 'NumOT')]

In [29]:
test = pd.read_csv(os.path.join('2019.csv'))
test.head()

,Season,DayNum,outcome,TeamID,Score,NumOT,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2019,1,1,1104,82,0,27,55,9,24,19,28,16,29,17,20,3,6,24
1,2019,1,1,1113,102,0,33,78,10,26,26,44,20,38,11,17,3,2,26
2,2019,1,1,1119,73,0,22,51,9,30,20,29,6,22,14,6,5,1,22
3,2019,1,1,1120,101,0,38,74,18,38,7,14,19,23,24,13,9,8,21
4,2019,1,1,1123,86,0,32,65,7,19,15,20,16,26,14,9,7,5,18


In [30]:
test.isnull().sum()

Season     0
DayNum     0
outcome    0
TeamID     0
Score      0
NumOT      0
FGM        0
FGA        0
FGM3       0
FGA3       0
FTM        0
FTA        0
OR         0
DR         0
Ast        0
TO         0
Stl        0
Blk        0
PF         0
dtype: int64

In [31]:
# X = test.drop(["outcome", "wpf"], axis=1)
X = test.drop("outcome", axis=1)
y = test["outcome"]
print(X.shape, y.shape)

(10926, 18) (10926,)


In [32]:
test_predictions = classifier.predict(X)
print(f"First 10 Predictions:   {test_predictions[:10]}")
print(f"First 10 Actual labels: {y[:10].tolist()}")

First 10 Predictions:   [1 1 1 1 1 1 1 1 1 1]
First 10 Actual labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
print(f"Testing Data Score: {classifier.score(X, y)}")

Testing Data Score: 0.8250045762401611


In [34]:
pd.DataFrame({"Prediction": test_predictions, "Actual": y}).reset_index(drop=True)

,Prediction,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [35]:
temp2=pd.DataFrame({"Prediction": test_predictions, "Actual": y,"TeamID":X["TeamID"].tolist()}).reset_index(drop=True)
output_file2="prediction_for_2019.csv"
temp2.to_csv(output_file2,index=False)

In [36]:
print(f"Testing Data Score: {classifier.score(X, y)}")

Testing Data Score: 0.8250045762401611


In [37]:
# Calculate the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=['Loss', 'Won']))

              precision    recall  f1-score   support

        Loss       0.88      0.89      0.89      8052
         Won       0.89      0.88      0.89      8051

   micro avg       0.89      0.89      0.89     16103
   macro avg       0.89      0.89      0.89     16103
weighted avg       0.89      0.89      0.89     16103



In [38]:
# Calculate the classification report
from sklearn.metrics import classification_report
print(classification_report(y, test_predictions,
                            target_names=['Loss', 'Won']))

              precision    recall  f1-score   support

        Loss       0.79      0.88      0.83      5463
         Won       0.87      0.77      0.81      5463

   micro avg       0.83      0.83      0.83     10926
   macro avg       0.83      0.83      0.82     10926
weighted avg       0.83      0.83      0.82     10926

